In [179]:
import pandas as pd 
from datetime import datetime

In [180]:
#Jobs file reading
df = pd.read_csv('glassdoor_jobs.csv', index_col = 0) 
#Duplicates delete
df.drop_duplicates(inplace=True)

In [181]:
#Job title cleaning
df.dropna(subset='job title', inplace=True)

In [182]:
#Fix seniority
df['job title'] = df['job title'].apply(lambda x : x.replace('Sr.', 'Senior').replace('Sr', 'Senior') \
    if (x.startswith('Sr.') or x.startswith('Sr')) else x)

df['job title'] = df['job title'].apply(lambda x : x.replace('Jr.', 'Junior').replace('Jr', 'Junior') \
    if (x.startswith('Jr.') or x.startswith('Jr')) else x)

In [183]:
#Salary parsing
df.dropna(subset=['salary estimate'], inplace=True)
df = df[df['salary estimate']!= '-1']
df = df[~(df['salary estimate'].str.contains('Per Hour'))]
salary = df['salary estimate'].apply(lambda x : x.split('(')[0] if '(' in x else x.split(':')[1])
salary = salary.apply(lambda x : x.replace('K', '').replace('$', ''))
salary_min = salary.apply(lambda x : x.split('-')[0] if '-' in x else x).astype('int')
salary_max = salary.apply(lambda x : x.split('-')[1] if '-' in x else x).astype('int')
df['salary estimate'] = (salary_min + salary_max)/2

In [184]:
df.dropna(subset=['location'], inplace=True)
#Company name text only
def is_float(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

df['company'] = df['company'].apply(lambda x : str(x)[:-3] if is_float(str(x)[-3:]) else x)

In [185]:
#State codes df
df = df[(df['location']!= 'United States') & (df['location'] !='Pennsylvania')]
df ['state code'] = df['location'].apply(lambda x : 'Remote' if x.lower()=='remote' else x.split(',')[-1])

In [186]:
#Age of the company
df['company_age'] = datetime.now().year-df['company_founded']

In [187]:
#Fix job description
df['job description'] = df['job description'] .apply(lambda x : x.replace('\r\n',''))
#Job description parsing (python, java, scala, SQL, etc.)
def set_score(tech_list):
    for tech in tech_list:
        df[tech] = df['job description'].apply(lambda x : 1 if tech in str(x).lower() else 0)

set_score(['python', 'java', 'scala', 'hadoop', 'sql', 'spark', 'aws', 'azure', 'gcp', 'airflow',
           'snowflake', 'git'])


In [188]:
df.loc[:, 'python':'git'].agg(sum)

python       260
java         100
scala        104
hadoop        82
sql          255
spark        111
aws          110
azure         37
gcp           17
airflow       12
snowflake     17
git          109
dtype: int64

In [189]:
#data_score = df.loc[:, 'python':'git'].agg(sum)
df.to_csv('glassdoor_jobs_cleaned.csv', index=False)